Есть группа словоформ, которая включает в себя слова с разным смыслом. Напишите код на python, который поможет разложить эти словоформы на смысловые группы. Объясните, почему эти слова оказались в одной группе

In [ ]:
!pip install -q pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 10.7 MB/s 


In [ ]:
import pymorphy2, pymorphy2_dicts_ru
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
words = 'роивший,роите,роил,роила,роись,рылами,рылах,рыле,роилась,роившись,роившийся,рылом,рылу,роивши,рыв,ройте,роят,роенные,роили,рывши,роило,роим,роющего,роимый,роящий,роющее,роенный,роющие,роющий,роющим,роющих,рыт,рытым,рытая,рыто,рытого,рытое,рытому,рыты,рытые,роенным,роит,роились,роилось,роился,рытый,рылось,роен,рыло,роить,роенными,роена,роенная,роенного,роенное,роющийся,роетесь,роется,роешься,рывший,рыл,роев,роем,рои,рою,роя,роям,роями,роях,рое,рылся,роют,рыть,рыться,роемый,роет,ройся,ройтесь,роюсь,роются,роющихся,рытых,роясь,рывшийся,рывшись,рыла,рылась,рыли,рылись,роемся,рытыми,роенной,роенных,роено,роимся,роитесь,роится,роете,роишь,роиться,роишься,роятся,роящиеся,роящийся,роящихся,роив,роешь,роены,рылам,рой'

Токенизация:

In [ ]:
words = words.replace(',', ' ')

In [ ]:
tokens = word_tokenize(words)

In [ ]:
morph = pymorphy2.MorphAnalyzer()

Лемматизация и группировка

Вариант 1, где учитываются все возможные начальные формы, -- более точный, но одно слово может относиться к нескольким группам.

In [ ]:
inf_mask = []
for word in tokens:
  p = morph.normal_forms(word)
  inf_mask.append(p)

In [ ]:
df1 = pd.DataFrame(
    {'tokens': tokens,
     'inf_mask': inf_mask,
    })
df1

,tokens,inf_mask
0,роивший,[роить]
1,роите,[роить]
2,роил,[роить]
3,роила,[роить]
4,роись,[роиться]
...,...,...
105,роив,[роить]
106,роешь,[рыть]
107,роены,[роить]
108,рылам,[рыло]


In [ ]:
df1 = df1.explode('inf_mask')

In [ ]:
morph_groups = df1.groupby('inf_mask')['tokens'].apply(list)
morph_groups

inf_mask
роить      [роивший, роите, роил, роила, роивши, роят, ро...
роиться    [роись, роилась, роившись, роившийся, роились,...
рой        [роев, роем, рои, рою, роя, роям, роями, роях,...
ройный                                                [роен]
рыло       [рылами, рылах, рыле, рылом, рылу, рыло, рыл, ...
рытый      [рыт, рытым, рытая, рыто, рытого, рытое, рытом...
рыть       [рыв, ройте, рывши, роющего, роющее, роющие, р...
рыться     [рылось, роющийся, роетесь, роется, роешься, р...
Name: tokens, dtype: object

In [ ]:
morph_groups.keys()

Index(['роить', 'роиться', 'рой', 'ройный', 'рыло', 'рытый', 'рыть', 'рыться'], dtype='object', name='inf_mask')

In [ ]:
'роюсь' in morph_groups['рыться'], 'роюсь' in morph_groups['роиться']

(True, True)

Вариант 2, где учитывается только наиболее вероятная начальная форма, -- менее точный, но более однозначный.

In [ ]:
inf_mask1 = []
for word in tokens:
  p = morph.normal_forms(word)[0]
  inf_mask1.append(p)

In [ ]:
df = pd.DataFrame(
    {'tokens': tokens,
     'inf_mask': inf_mask1,
    })
df

,tokens,inf_mask
0,роивший,роить
1,роите,роить
2,роил,роить
3,роила,роить
4,роись,роиться
...,...,...
105,роив,роить
106,роешь,рыть
107,роены,роить
108,рылам,рыло


In [ ]:
morpho_groups = df.groupby('inf_mask')['tokens'].apply(list)
morpho_groups

inf_mask
роить      [роивший, роите, роил, роила, роивши, роят, ро...
роиться    [роись, роилась, роившись, роившийся, роились,...
рой        [роев, роем, рои, рою, роя, роям, роями, роях,...
ройный                                                [роен]
рыло       [рылами, рылах, рыле, рылом, рылу, рыло, рыл, ...
рытый      [рыт, рытым, рытая, рыто, рытого, рытое, рытом...
рыть       [рыв, ройте, рывши, роющего, роющее, роющие, р...
рыться     [рылось, роющийся, роетесь, роется, роешься, р...
Name: tokens, dtype: object

In [ ]:
morpho_groups.keys()

Index(['роить', 'роиться', 'рой', 'ройный', 'рыло', 'рытый', 'рыть', 'рыться'], dtype='object', name='inf_mask')

In [ ]:
'роюсь' in morpho_groups['рыться'], 'роюсь' in morpho_groups['роиться']

(False, True)

Можно немного сократить количество групп с помощью стемминга нормализованных форм слов (используются данные из варианта 1).

In [ ]:
from nltk.stem import SnowballStemmer

In [ ]:
stem_list = morph_groups.keys()
ss = SnowballStemmer(language='russian')
stems = list(map(ss.stem, stem_list))

In [ ]:
morph_groups1 = morph_groups.to_frame()
morph_groups1.reset_index(inplace=True)

In [ ]:
morph_groups1.insert(loc=0, column='stems', value=stems)

In [ ]:
morph_groups1

,stems,inf_mask,tokens
0,ро,роить,"[роивший, роите, роил, роила, роивши, роят, ро..."
1,ро,роиться,"[роись, роилась, роившись, роившийся, роились,..."
2,ро,рой,"[роев, роем, рои, рою, роя, роям, роями, роях,..."
3,ройн,ройный,[роен]
4,рыл,рыло,"[рылами, рылах, рыле, рылом, рылу, рыло, рыл, ..."
5,рыт,рытый,"[рыт, рытым, рытая, рыто, рытого, рытое, рытом..."
6,рыт,рыть,"[рыв, ройте, рывши, роющего, роющее, роющие, р..."
7,рыт,рыться,"[рылось, роющийся, роетесь, роется, роешься, р..."


In [ ]:
morph_groups1['tokens'] = [', '.join(map(str, l)) for l in morph_groups1['tokens']]

In [ ]:
print(morph_groups1.to_string())

  stems inf_mask                                                                                                                                                                                                                                             tokens
0    ро    роить               роивший, роите, роил, роила, роивши, роят, роенные, роили, роило, роим, роимый, роящий, роенный, роенным, роит, роен, роить, роенными, роена, роенная, роенного, роенное, рои, рою, роя, роенной, роенных, роено, роишь, роив, роены
1    ро  роиться                                                                                       роись, роилась, роившись, роившийся, роились, роилось, роился, роюсь, роясь, роимся, роитесь, роится, роиться, роишься, роятся, роящиеся, роящийся, роящихся
2    ро      рой                                                                                                                                                                                             роев, роем, рои

Таким образом, получаются следующие семантические группы:
*   с основой "ро", связанные с роем и роением
*   с основой "рыт", связанные с рытьем
*   с основой "рыл" -- склонение слова "рыло"
*   с основой "ройн", похожая на выброс вследствие попытки лемматизатора распознать неизвестное слово "роен".